## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716546824066, experiment_id='1', last_update_time=1716546824066, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716542919366, experiment_id='0', last_update_time=1716542919366, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name="my-cool-experiments")

'3'

Let's check the latest versions for the experiment with id `1`...

In [14]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [15]:
# runs

In [16]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 7fdb8da0da054dad8ccc9888fafd81fb, rmse: 6.3184
run id: 4b6063e2230340d8a8438fb88cbfd015, rmse: 6.3184
run id: 68e97a6194fd4f81a310cf843c574605, rmse: 6.7423
run id: c0051c3ce6db4c60bd2b014e25d77c4f, rmse: 6.9046
run id: ae5f453fcc7542aba86146507d90e42c, rmse: 6.9211


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [17]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [18]:
# run id: 7fdb8da0da054dad8ccc9888fafd81fb, rmse: 6.3184
# run id: 4b6063e2230340d8a8438fb88cbfd015, rmse: 6.3184
# run id: 68e97a6194fd4f81a310cf843c574605, rmse: 6.7423
# run id: c0051c3ce6db4c60bd2b014e25d77c4f, rmse: 6.9046
# run id: ae5f453fcc7542aba86146507d90e42c, rmse: 6.9211


run_id = "68e97a6194fd4f81a310cf843c574605"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '12' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716696333441, current_stage='None', description=None, last_updated_timestamp=1716696333441, name='nyc-taxi-regressor', run_id='68e97a6194fd4f81a310cf843c574605', run_link=None, source='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/68e97a6194fd4f81a310cf843c574605/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=12>

In [19]:
client.search_registered_models

<bound method MlflowClient.search_registered_models of <mlflow.tracking.client.MlflowClient object at 0x15a803730>>

In [20]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 12, stage: None
version: 4, stage: Production
version: 11, stage: Staging


/var/folders/dl/74t5rlpn7yg9n8p83fsrhfrh0000gn/T/ipykernel_15685/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [21]:
model_version = 12
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/dl/74t5rlpn7yg9n8p83fsrhfrh0000gn/T/ipykernel_15685/2013686150.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716696333441, current_stage='Staging', description=None, last_updated_timestamp=1716696502060, name='nyc-taxi-regressor', run_id='68e97a6194fd4f81a310cf843c574605', run_link=None, source='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/68e97a6194fd4f81a310cf843c574605/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=12>

In [22]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716696333441, current_stage='Staging', description='The model version 12 was transitioned to Staging on 2024-05-26', last_updated_timestamp=1716696550090, name='nyc-taxi-regressor', run_id='68e97a6194fd4f81a310cf843c574605', run_link=None, source='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/68e97a6194fd4f81a310cf843c574605/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=12>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [23]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [24]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

/var/folders/dl/74t5rlpn7yg9n8p83fsrhfrh0000gn/T/ipykernel_15685/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [30]:
run_id

'68e97a6194fd4f81a310cf843c574605'

In [25]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [26]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [31]:
X_test = preprocess(df, dv)

In [32]:
target = "duration"
y_test = df[target].values

In [34]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 311 ms, sys: 55.9 ms, total: 367 ms
Wall time: 455 ms


{'rmse': 6.659623830022514}

In [35]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 97.8 ms, sys: 2.89 ms, total: 101 ms
Wall time: 102 ms


/opt/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


{'rmse': 6.659623830022514}

In [36]:
client.transition_model_version_stage(
    name=model_name,
    version=12,
    stage="Production",
    archive_existing_versions=True
)

/var/folders/dl/74t5rlpn7yg9n8p83fsrhfrh0000gn/T/ipykernel_15685/127275438.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716696333441, current_stage='Production', description='The model version 12 was transitioned to Staging on 2024-05-26', last_updated_timestamp=1716698573528, name='nyc-taxi-regressor', run_id='68e97a6194fd4f81a310cf843c574605', run_link=None, source='/Users/son/Documents/GitHub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/68e97a6194fd4f81a310cf843c574605/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=12>